## Practice

1. Input images from webcam.
2. Use <b>MediaPipe()</b> to peform face detection.
3. Find the coordinates of 6 FaceKeyPoint: (RIGHT_EYE,LEFT_EYE,NOSE_TIP,MOUTH_CENTER,RIGHT_EAR_TRAGION,LEFT_EAR_TRAGION)
4. Draw a triangle using three points: RIGHT_EYE, LEFT_EYE, MOUTH_CENTER
5. Use <i>cv2.arrowedLine()</i> to indicate the normal direction of the face. (hint: a vector from the center of the triangle to the NOSE_TIP)
6. Show the output image.
7. Upload your Jupyter code file (*.ipynb)

In [3]:
import cv2
import numpy as np
import mediapipe as mp 

In [2]:
def list_available_cameras(max_index):
    available_cameras = []
    for i in range(max_index):
        cap = cv2.VideoCapture(i)
        
        if cap.isOpened():
            available_cameras.append(i)
            cap.release()               
    return available_cameras

max_camera_index = 10
available_cameras = list_available_cameras(max_camera_index)

print("Available Cameras:", available_cameras)

[ WARN:0@12.282] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@12.399] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range


Available Cameras: [0, 2]


[ WARN:0@12.487] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video3): can't open camera by index
[ERROR:0@12.488] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@12.489] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video4): can't open camera by index
[ERROR:0@12.492] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@12.492] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video5): can't open camera by index
[ERROR:0@12.494] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@12.494] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video6): can't open camera by index
[ERROR:0@12.496] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@12.496] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video7): can't open camera by index
[ERROR:0@12.498] global obsensor_uvc_stream_channel.cpp:159 getStreamChan

In [4]:
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection()


libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: 

In [6]:
cap = cv2.VideoCapture(0)

while cap.isOpened():

    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    results = face_detection.process(rgb_frame)
    
    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                   int(bboxC.width * iw), int(bboxC.height * ih)
            
            cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[0]+bbox[2], bbox[1]+bbox[3]), (0, 255, 0), 2)
            
            landmarks = detection.location_data.relative_keypoints
            landmarks_dict = {}
            for idx, landmark in enumerate(landmarks):
                x = int(landmark.x * iw)
                y = int(landmark.y * ih)
                landmarks_dict[idx] = (x, y)
            
                cv2.circle(frame, (x, y), 2, (0, 255, 255), -1)
            
            right_eye = landmarks_dict[0]
            left_eye = landmarks_dict[1]
            nose_tip = landmarks_dict[2]
            mouth_center = landmarks_dict[3]
            right_ear_tragion = landmarks_dict[4]
            left_ear_tragion = landmarks_dict[5]
            
            cv2.line(frame, right_eye, left_eye, (255, 0, 0), 2)
            cv2.line(frame, left_eye, mouth_center, (255, 0, 0), 2)
            cv2.line(frame, mouth_center, right_eye, (255, 0, 0), 2)
            
            center_x = (right_eye[0] + left_eye[0] + mouth_center[0]) // 3
            center_y = (right_eye[1] + left_eye[1] + mouth_center[1]) // 3
            cv2.arrowedLine(frame, (center_x, center_y), nose_tip, (0, 255, 255), 2)

    cv2.imshow('Face Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()